In [ ]:
import czifile
import numpy as np

import skimage.io
import skimage.filters
import skimage.feature
import skimage.morphology

import scipy

import bebi103

import holoviews as hv
hv.extension('bokeh')
import colorcet

import bokeh.io
bokeh.io.output_notebook()

In [ ]:
def show_two_ims(
    im_1,
    im_2,
    titles=[None, None],
    interpixel_distances=[0.13, 0.13],
    cmap=None,
):
    """Convenient function for showing two images side by side."""
    p_1 = bebi103.image.imshow(
        im_1,
        frame_height=200,
        title=titles[0],
        cmap=cmap,
        interpixel_distance=interpixel_distances[0],
        length_units="µm",
    )
    p_2 = bebi103.image.imshow(
        im_2,
        frame_height=200,
        title=titles[1],
        cmap=cmap,
        interpixel_distance=interpixel_distances[1],
        length_units="µm",
    )
    p_2.x_range = p_1.x_range
    p_2.y_range = p_1.y_range

    return bokeh.layouts.gridplot([p_1, p_2], ncols=2)

def dapi_sort(image):
    return image[0,0,0,0,:,:,:,0]

def image_reducer(filepath, zoom):
    img = czifile.imread(filepath)
    img = dapi_sort(img)
    img = np.sum(img, 0)
    
    return img[zoom]


def zero_crossing_filter(im, thresh):
    """
    Returns image with 1 if there is a zero crossing and 0 otherwise.

    thresh is the the minimal value of the gradient, as computed by Sobel
    filter, at crossing to count as a crossing.
    """
    # Square structuring element
    selem = skimage.morphology.square(3)

    # Do max filter and min filter
    im_max = scipy.ndimage.filters.maximum_filter(im, footprint=selem)
    im_min = scipy.ndimage.filters.minimum_filter(im, footprint=selem)

    # Compute gradients using Sobel filter
    im_grad = skimage.filters.sobel(im)

    # Return edges
    return ( (  ((im >= 0) & (im_min < 0))
              | ((im <= 0) & (im_max > 0)))
            & (im_grad >= thresh) )
        
zoom1 = np.s_[1500:3000, :1500]

In [ ]:
im1 = image_reducer('20.02.15_DAPI-seg-1.czi', zoom1)

In [ ]:
im1.shape

In [ ]:
hv.Image(im1
).opts(
    height = 500,
    width = 500)

In [ ]:
# Convert the uneven image to floating point
im_float = im1 / im1.max()

# Make the structuring element
selem = skimage.morphology.square(25)

# Perform the median filter
im1_filt = skimage.filters.median(im_float, selem)

In [ ]:
bokeh.io.show(show_two_ims(im1, im1_filt))

p_2 = bebi103.image.imshow(
    im1_filt,
    cmap=colorcet.coolwarm,
    frame_height=300,
    interpixel_distance= 0.12,
    length_units="µm",
)
bokeh.io.show(p_2)

In [ ]:
# Convert image to float
im_float = (im1_filt.astype(float) - im1_filt.min()) / (im1_filt.max() - im1_filt.min())

# Apply the Sobel filter to find the gradient
im_grad = skimage.filters.sobel(im_float)

# Look at gradient image
#bokeh.io.show(
#    show_two_ims(im_float, im_grad, titles=["original", "sobel filtered"])
#)

In [ ]:
# Do Canny edge detection on image, use sigma = 1.4
im_edge = skimage.feature.canny(im_float, 0.3)

# Show the result
bokeh.io.show(show_two_ims(im_float, im_edge, titles=["original", "edges"]))

In [ ]:
# Close the edge image
selem = skimage.morphology.disk(15)
im_edge_closed = skimage.morphology.binary_closing(im_edge, selem)

# Fill these holes
im_bw = scipy.ndimage.morphology.binary_fill_holes(im_edge_closed)

# Check out results
bokeh.io.show(
    show_two_ims(im_float, im_bw, titles=["original", "segmented"])
)